In this notebook we classify fake news with a transformer model.<br>
<br>
We have already encoded the fake news train & test datasets in the **Fake news encoding** notebook,<br>
here we will:
- Add positional encodings to the data.
- Initialize a transformer model.
- Train the model on the train data.
- Evaluate the models predictions on the test data.

We show several versions of transfomer models and encodings of the datasets, in each version we try different approaches of encodings and model-building.


A guide to Transformer models that we used:<br>
https://keras.io/examples/nlp/text_classification_with_transformer/

# Learning with Bag-of-words encodings

In this section, we use the corpus BOW encodings we created in the **Fake news encoding** notebook as data for our model.<br>
<br>
The corpus is encoded in the following manner- Each document is converted to an array of numbers, each number represents a matching word in the document. there are **56668** words altogether.<br>
<br>
We will build, fit and evaluate three different models:
1. Transformer model that **multiclass-classifies** **6** categories.
2. Transformer model that **binary-classifies**, fitted and tested only on documents that are labeled as **'true'**, **'false'**.
3. Transformer model that **binary-classifies**, fitted and tested on all documents but the labels will be merged to two categories - **'true'**, **'false'**.

## Model 1 - Multiclass classification

In [ ]:
import pandas as pd
import numpy as np
import io
import time
from tqdm import tqdm
from math import sin, cos

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Get datasets

In [ ]:
#train
num_splits = 7
train = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_{i}.csv')
  train = train.append(temp)

#train encoded
num_splits = 4
train_encoded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_encoded_v3_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_encoded_v3_{i}.csv')
  train_encoded = train_encoded.append(temp)

#test
test = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset-train/main/liar_test.csv')

#test encoded
test_encoded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/test_encoded_v3.csv')

In [ ]:
train_encoded_lst = list(train_encoded['full_text_encoded'])
test_encoded_lst = list(test_encoded['full_text_encoded'])

# documents encodings are saved as strings
# use eval to return it to actual encodings (list of numbers)
for i, doc in enumerate(train_encoded_lst):
  train_encoded_lst[i] = eval(doc)

for i, doc in enumerate(test_encoded_lst):
  test_encoded_lst[i] = eval(doc)

### Get train and test labels One-hot encodings

In [ ]:
train_labels_oh = pd.get_dummies(train['label-liar'])
test_labels_oh = pd.get_dummies(test['label-liar'])

train_labels = np.asarray(train_labels_oh).tolist()
test_labels = np.asarray(test_labels_oh).tolist()

### Truncate/pad sequences to a *max sequence length*
First we need to determine the value of *max sequence length*.<br>
Find minimum, maximum and average sequence length.

In [ ]:
doc_len = len(train_encoded_lst[0])
min_len = doc_len
max_len = doc_len
avg_len = doc_len
for doc in train_encoded_lst:
  doc_len = len(doc)

  if min_len > doc_len:
    min_len = doc_len

  elif max_len < doc_len:
    max_len = doc_len
  
  avg_len = (avg_len+doc_len)/2

print('min sequence length =', min_len)
print('max sequence length =', max_len)
print('avrage sequence length =', avg_len)

min sequence length = 33
max sequence length = 1616
avrage sequence length = 466.2858263541781


Decision: Our model will use a *max sequence length* of 500

In [ ]:
def pad_list(lst, value=0, size=0):
  count=0
  while count < size:
    lst.append(value)
    count += 1

In [ ]:
maxlen = 500 # max sequence length

train_docs = []
for doc in tqdm(train_encoded_lst, position=0, leave=True):
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, size=pad_size) # pad end of seq with zeros
  train_docs.append(truncated_doc)

test_docs = []
for doc in tqdm(test_encoded_lst, position=0, leave=True):
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, size=pad_size) # pad end of seq with zeros
  test_docs.append(truncated_doc)

100%|██████████| 1266/1266 [00:00<00:00, 44658.91it/s]


In [ ]:
# check that all sequence lengths' are the same
doc_lens = set()
for doc in train_docs:
  doc_lens.add(len(doc))

for doc in test_docs:
  doc_lens.add(len(doc))

print(doc_lens)

{500}


### Transformer blocks and positional embeddings

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
def positional_embeddings(maxlen):
  pe_lst = []
  for pos in range(maxlen):
    pe_lst.append(sin(pos))

  return pe_lst

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x, maxlen):
        positions = tf.convert_to_tensor(positional_embeddings(maxlen))
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

### Build model and fit

In [ ]:
vocab_size = 56668 # taken from 'Fake news embedding' notebook
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs, maxlen)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(6, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
x_train = tf.convert_to_tensor(train_docs)
y_train = tf.convert_to_tensor(train_labels)

In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=["categorical_accuracy", tf.keras.metrics.Recall()])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_split=0.2, shuffle=True
)

Epoch 1/2
377/377 [==============================] - 233s 614ms/step - loss: 1.7644 - categorical_accuracy: 0.2060 - recall_20: 6.4111e-04 - val_loss: 1.6524 - val_categorical_accuracy: 0.2783 - val_recall_20: 0.0442
Epoch 2/2
377/377 [==============================] - 233s 618ms/step - loss: 1.5899 - categorical_accuracy: 0.3064 - recall_20: 0.0440 - val_loss: 1.5615 - val_categorical_accuracy: 0.3318 - val_recall_20: 0.0571


### Evaluate model on test

In [ ]:
x_test = tf.convert_to_tensor(test_docs)
y_test = tf.convert_to_tensor(test_labels)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=32)

40/40 [==============================] - 7s 178ms/step - loss: 1.5917 - categorical_accuracy: 0.2930 - recall_20: 0.0134


## Model 2 - Binary classification, only 'true', 'false'

In [ ]:
import pandas as pd
import numpy as np
import io
import time
from tqdm import tqdm
from math import sin, cos

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Get datasets

In [ ]:
#train
num_splits = 7
train = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_{i}.csv')
  train = train.append(temp)

#train encoded
num_splits = 4
train_encoded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_encoded_v3_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_encoded_v3_{i}.csv')
  train_encoded = train_encoded.append(temp)

#test
test = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset-train/main/liar_test.csv')

#test encoded
test_encoded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/test_encoded_v3.csv')

### Filter documents and labels where labels are binary ('true' or 'false')

In [ ]:
train_labels_lst = train['label-liar'].tolist()
train_encoded_lst = train_encoded['full_text_encoded'].tolist()

test_labels_lst = test['label-liar'].tolist()
test_encoded_lst = test_encoded['full_text_encoded'].tolist()

In [ ]:
is_binary_train_index = lambda i: True if train_labels_lst[i] in ('true', 'false') else False

is_binary_test_index = lambda i: True if test_labels_lst[i] in ('true', 'false') else False

In [ ]:
train_labels_binary = []
train_encoded_binary = []

test_labels_binary = []
test_encoded_binary = []

for i in range(len(train_labels_lst)):
  if is_binary_train_index(i):
    train_labels_binary.append(train_labels_lst[i])
    train_encoded_binary.append(train_encoded_lst[i])

for i in range(len(test_labels_lst)):
  if is_binary_test_index(i):
    test_labels_binary.append(test_labels_lst[i])
    test_encoded_binary.append(test_encoded_lst[i])

# documents encodings are saved as strings
# use eval to return it to actual encodings (list of numbers)
for i, doc in enumerate(train_encoded_binary):
  train_encoded_binary[i] = eval(doc)

for i, doc in enumerate(test_encoded_binary):
  test_encoded_binary[i] = eval(doc)

### Get train and test labels One-hot encodings

In [ ]:
train_labels_binary_df = pd.DataFrame({'label-liar': train_labels_binary})
test_labels_binary_df = pd.DataFrame({'label-liar': test_labels_binary})

train_labels_oh = pd.get_dummies(train_labels_binary_df['label-liar'])
test_labels_oh = pd.get_dummies(test_labels_binary_df['label-liar'])

train_labels = np.asarray(train_labels_oh).tolist()
test_labels = np.asarray(test_labels_oh).tolist()

### Truncate/pad sequences to a max sequence length
First we need to determine the value of max sequence length.<br>
Find minimum, maximum and average sequence length

In [ ]:
doc_len = len(train_encoded_binary[0])
min_len = doc_len
max_len = doc_len
avg_len = doc_len
for doc in train_encoded_binary:
  doc_len = len(doc)

  if min_len > doc_len:
    min_len = doc_len

  elif max_len < doc_len:
    max_len = doc_len
  
  avg_len = (avg_len+doc_len)/2

print('min sequence length =', min_len)
print('max sequence length =', max_len)
print('avrage sequence length =', avg_len)

min sequence length = 38
max sequence length = 1616
avrage sequence length = 393.13873830976917


Decision: Our model will use a *max sequence length* of 500

In [ ]:
def pad_list(lst, value=0, size=0):
  count=0
  while count < size:
    lst.append(value)
    count += 1

In [ ]:
maxlen = 500 # max sequence length

train_docs = []
for doc in tqdm(train_encoded_binary, position=0, leave=True):
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, size=pad_size) # pad end of seq with zeros
  train_docs.append(truncated_doc)

test_docs = []
for doc in tqdm(test_encoded_binary, position=0, leave=True):
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, size=pad_size) # pad end of seq with zeros
  test_docs.append(truncated_doc)

100%|██████████| 457/457 [00:00<00:00, 52527.94it/s]


In [ ]:
# check that all sequence lengths' are the same
doc_lens = set()
for doc in train_docs:
  doc_lens.add(len(doc))

for doc in train_docs:
  doc_lens.add(len(doc))

print(doc_lens)

{500}


### Transformer blocks and positional embeddings

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
def positional_embeddings(maxlen):
  pe_lst = []
  for pos in range(maxlen):
    pe_lst.append(sin(pos))

  return pe_lst

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x, maxlen):
        # positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = tf.convert_to_tensor(positional_embeddings(maxlen))
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

### Build model and fit

In [ ]:
vocab_size = 56668 # taken from 'Fake news embedding' notebook
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs, maxlen)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
x_train = tf.convert_to_tensor(train_docs)
y_train = tf.convert_to_tensor(train_labels)

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["binary_accuracy", tf.keras.metrics.Recall()])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=4, validation_split=0.2, shuffle=True
)

Epoch 1/4
134/134 [==============================] - 84s 613ms/step - loss: 0.6728 - binary_accuracy: 0.5896 - recall_21: 0.5896 - val_loss: 0.6239 - val_binary_accuracy: 0.6492 - val_recall_21: 0.6492
Epoch 2/4
134/134 [==============================] - 82s 609ms/step - loss: 0.5881 - binary_accuracy: 0.6694 - recall_21: 0.6694 - val_loss: 0.6350 - val_binary_accuracy: 0.6426 - val_recall_21: 0.6426
Epoch 3/4
134/134 [==============================] - 83s 617ms/step - loss: 0.3781 - binary_accuracy: 0.8416 - recall_21: 0.8416 - val_loss: 0.5850 - val_binary_accuracy: 0.7298 - val_recall_21: 0.7298
Epoch 4/4
134/134 [==============================] - 83s 622ms/step - loss: 0.2067 - binary_accuracy: 0.9230 - recall_21: 0.9230 - val_loss: 0.7676 - val_binary_accuracy: 0.7186 - val_recall_21: 0.7186


### Evaluate model on test

In [ ]:
x_test = tf.convert_to_tensor(test_docs)
y_test = tf.convert_to_tensor(test_labels)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=32)

15/15 [==============================] - 3s 178ms/step - loss: 0.8222 - binary_accuracy: 0.6958 - recall_21: 0.6958


## Model 3 - Binary classification, merged classes

In [ ]:
import pandas as pd
import numpy as np
import io
import time
from tqdm import tqdm
from math import sin, cos

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Get datasets

In [ ]:
#train
num_splits = 7
train = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_{i}.csv')
  train = train.append(temp)

#train encoded
num_splits = 4
train_encoded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_encoded_v3_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_encoded_v3_{i}.csv')
  train_encoded = train_encoded.append(temp)

#test
test = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset-train/main/liar_test.csv')

#test encoded
test_encoded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/test_encoded_v3.csv')

In [ ]:
train_encoded_lst = train_encoded['full_text_encoded'].tolist()
test_encoded_lst = test_encoded['full_text_encoded'].tolist()

# documents encodings are saved as strings
# use eval to return it to actual encodings (list of numbers)
for i, doc in enumerate(train_encoded_lst):
  train_encoded_lst[i] = eval(doc)

for i, doc in enumerate(test_encoded_lst):
  test_encoded_lst[i] = eval(doc)

### Merge labels
- ('true', 'mostly-true', 'half-true') => 'true'
- ('false', 'pants-fire', 'barely-true') => 'false'

In [ ]:
train['label-liar'].unique()

array(['barely-true', 'pants-fire', 'half-true', 'mostly-true', 'true',
       'false'], dtype=object)

In [ ]:
merge_labels = lambda label: 'true' if label in ('true', 'mostly-true', 'half-true') else 'false'

In [ ]:
train_labels_lst = train['label-liar'].tolist()
test_labels_lst = test['label-liar'].tolist()

train_labels_merged = []
test_labels_merged = []

for label in train_labels_lst:
  train_labels_merged.append(merge_labels(label))

for label in test_labels_lst:
  test_labels_merged.append(merge_labels(label))

### Get train and test labels One-hot encodings

In [ ]:
train_labels_merged_df = pd.DataFrame({'label-liar': train_labels_merged})
test_labels_merged_df = pd.DataFrame({'label-liar': test_labels_merged})

train_labels_oh = pd.get_dummies(train_labels_merged_df['label-liar'])
test_labels_oh = pd.get_dummies(test_labels_merged_df['label-liar'])

train_labels = np.asarray(train_labels_oh).tolist()
test_labels = np.asarray(test_labels_oh).tolist()

### Truncate/pad sequences to a max sequence length
First we need to determine the value of max sequence length.<br>
Find minimum, maximum and average sequence length

In [ ]:
doc_len = len(train_encoded_lst[0])
min_len = doc_len
max_len = doc_len
avg_len = doc_len
for doc in train_encoded_lst:
  doc_len = len(doc)

  if min_len > doc_len:
    min_len = doc_len

  elif max_len < doc_len:
    max_len = doc_len
  
  avg_len = (avg_len+doc_len)/2

print('min sequence length =', min_len)
print('max sequence length =', max_len)
print('avrage sequence length =', avg_len)

min sequence length = 33
max sequence length = 1616
avrage sequence length = 466.2858263541781


Decision: Our model will use a *max sequence length* of 500

In [ ]:
def pad_list(lst, value=0, size=0):
  count=0
  while count < size:
    lst.append(value)
    count += 1

In [ ]:
maxlen = 500 # max sequence length

train_docs = []
for doc in train_encoded_lst:
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, size=pad_size) # pad end of seq with zeros
  train_docs.append(truncated_doc)

test_docs = []
for doc in test_encoded_lst:
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, size=pad_size) # pad end of seq with zeros
  test_docs.append(truncated_doc)

In [ ]:
# check that all sequence lengths' are the same
doc_lens = set()
for doc in train_docs:
  doc_lens.add(len(doc))

for doc in test_docs:
  doc_lens.add(len(doc))

print(doc_lens)

{500}


### Transformer blocks and positional embeddings

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
def positional_embeddings(maxlen):
  pe_lst = []
  for pos in range(maxlen):
    pe_lst.append(sin(pos))

  return pe_lst

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x, maxlen):
        # positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = tf.convert_to_tensor(positional_embeddings(maxlen))
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

### Build model and fit

In [ ]:
vocab_size = 56668 # taken from 'Fake news embedding' notebook
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs, maxlen)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
x_train = tf.convert_to_tensor(train_docs)
y_train = tf.convert_to_tensor(train_labels)

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["binary_accuracy", tf.keras.metrics.Recall()])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=1, validation_split=0.2, shuffle=True
)

377/377 [==============================] - 239s 628ms/step - loss: 0.6877 - binary_accuracy: 0.5449 - recall_22: 0.5449 - val_loss: 0.5836 - val_binary_accuracy: 0.6828 - val_recall_22: 0.6828


### Evaluate model on test

In [ ]:
x_test = tf.convert_to_tensor(test_docs)
y_test = tf.convert_to_tensor(test_labels)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=32)

40/40 [==============================] - 7s 184ms/step - loss: 0.6209 - binary_accuracy: 0.6532 - recall_22: 0.6532


# Learning with Bert's contextualized embeddings

In this section, we use the fake news embeddings we extracted from Bert as data for our model.<br>
We extracted two different embeddings:<br>
**The first**, embeddings Bert learned with *max sequence length* of 128.<br>
**The second**, embeddings Bert learned with *max sequence length* of 512 (the max *max sequence length* for bert).<br>
<br>
We will build, fit and evaluate three different models:
1. Transformer model that **multiclass-classifies** to **6** categories, fitted on **the first** embeddings.
2. Transformer model that **multiclass-classifies** to **6** categories, fitted on **the second** embeddings.
3. Transformer model that **binary-classifies**, fitted on **the second** embeddings.

## Model 1 - Multiclass-classification, *max sequence length* = 128

In [ ]:
import pandas as pd
import numpy as np
import io
import time
from tqdm import tqdm
from math import sin, cos

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Get datasets

In [ ]:
#train
num_splits = 7
train = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_{i}.csv')
  train = train.append(temp)

#train embedded
num_splits = 4
train_embedded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_embedded_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_embedded_{i}.csv')
  train_embedded = train_embedded.append(temp)

#test
test = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset-train/main/liar_test.csv')

#test embedded
test_embedded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/test_embedded.csv')

In [ ]:
train_embedded.head(3)

,full_text_embedded
0,"[[-0.39664504, -3.201748, -1.6849961, 0.911999..."
1,"[[-0.112607814, -2.6428027, -1.084317, 1.01488..."
2,"[[1.1767617, -1.7852252, -2.0916524, 0.0093422..."


In [ ]:
train_embedded_lst = train_embedded['full_text_embedded'].tolist()
test_embedded_lst = test_embedded['full_text_embedded'].tolist()

# documents embeddings are saved as strings
# use eval to return it to acctual embeddings (list of word embedding vectors)
for i, doc in enumerate(train_embedded_lst):
  train_embedded_lst[i] = eval(doc)

for i, doc in enumerate(test_embedded_lst):
  test_embedded_lst[i] = eval(doc)

### Get train and test labels One-hot encodings

In [ ]:
train_labels_oh = pd.get_dummies(train['label-liar'])
test_labels_oh = pd.get_dummies(test['label-liar'])

train_labels = np.asarray(train_labels_oh).tolist()
test_labels = np.asarray(test_labels_oh).tolist()

### Truncate/pad sequences to a *max sequence length*

First we need to determine the value of max sequence length.<br>
Find minimum, maximum and average sequence length.

In [ ]:
doc_len = len(train_embedded_lst[0])
min_len = doc_len
max_len = doc_len
avg_len = doc_len
for doc in train_embedded_lst:
  doc_len = len(doc)

  if min_len > doc_len:
    min_len = doc_len

  elif max_len < doc_len:
    max_len = doc_len
  
  avg_len = (avg_len+doc_len)/2

print('min sequence length =', min_len)
print('max sequence length =', max_len)
print('avrage sequence length =', avg_len)

min sequence length = 49
max sequence length = 109
avrage sequence length = 81.47448309689022


Decision: Our model will use a *max sequence length* of 80.

In [ ]:
def pad_list(lst, value=0, size=0):
  count=0
  while count < size:
    lst.append(value)
    count += 1

In [ ]:
maxlen = 80 # max sequence length

train_docs = []
for doc in train_embedded_lst:
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, value=[0]*5, size=pad_size) # pad end of seq with zeros
  train_docs.append(truncated_doc)

test_docs = []
for doc in test_embedded_lst:
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, value=[0]*5, size=pad_size) # pad end of seq with zeros
  test_docs.append(truncated_doc)

In [ ]:
# check that all sequence lengths' are the same
doc_lens = set()
for doc in train_docs:
  doc_lens.add(len(doc))

for doc in test_docs:
  doc_lens.add(len(doc))

print(doc_lens)

{80}


### Add positional embeddings

In [ ]:
def positional_embeddings(vector, dims):
  dim1, dim2 = dims
  pe = 0
  for pos in range(dim1):
    for i in range(dim2):

      if i%2 == 0: # is even
        pe = sin(pos/10000**(i/dim2))

      else: # is odd
        pe = cos(pos/10000**((i-1)/dim2))

      vector[pos][i] += pe

In [ ]:
word_size = 5 # each word is represented as a vector with dim = word_size

for vector in train_docs:
  positional_embeddings(vector, (maxlen, word_size))

for vector in test_docs:
  positional_embeddings(vector, (maxlen, word_size))

### Transformer block

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

### Build model and fit

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,5))
embedding_layer = layers.Dense(embed_dim) # (not realy an embedding_layer, regular fc layer)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(6, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
x_train = tf.convert_to_tensor(train_docs)
y_train = tf.convert_to_tensor(train_labels)

In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=["categorical_accuracy", tf.keras.metrics.Recall()])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_split=0.2, shuffle=True
)

Epoch 1/2
377/377 [==============================] - 25s 63ms/step - loss: 1.7809 - categorical_accuracy: 0.1985 - recall_23: 2.4773e-05 - val_loss: 1.7370 - val_categorical_accuracy: 0.2308 - val_recall_23: 0.0000e+00
Epoch 2/2
377/377 [==============================] - 24s 63ms/step - loss: 1.7439 - categorical_accuracy: 0.2171 - recall_23: 2.0010e-04 - val_loss: 1.7263 - val_categorical_accuracy: 0.2371 - val_recall_23: 0.0013


### Evaluate model on test

In [ ]:
x_test = tf.convert_to_tensor(test_docs)
y_test = tf.convert_to_tensor(test_labels)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=32)

40/40 [==============================] - 1s 20ms/step - loss: 1.7545 - categorical_accuracy: 0.2014 - recall_23: 0.0000e+00


## Model 2 - Multiclass-classification, *max sequence length* = 512

In [ ]:
import pandas as pd
import numpy as np
import io
import time
from tqdm import tqdm
from math import sin, cos

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Get datasets

In [ ]:
#train
num_splits = 7
train = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_{i}.csv')
  train = train.append(temp)

#train embedded
num_splits = 12
train_embedded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_embedded_v2_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_embedded_v2_{i}.csv')
  train_embedded = train_embedded.append(temp)

#test
test = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset-train/main/liar_test.csv')

#test embedded
test_embedded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/test_embedded.csv')

In [ ]:
train_embedded.head(3)

,full_text_embedded
0,"[[-0.37441736, -4.348481, -2.867022, -0.833148..."
1,"[[-1.0242298, -3.5270042, -1.9221768, 0.864446..."
2,"[[0.03841588, -3.0448728, -2.48761, 0.37428683..."


In [ ]:
train_embedded_lst = train_embedded['full_text_embedded'].tolist()
test_embedded_lst = test_embedded['full_text_embedded'].tolist()

# documents embeddings are saved as strings
# use eval to return it to acctual embeddings (list of word embedding vectors)
for i, doc in enumerate(train_embedded_lst):
  train_embedded_lst[i] = eval(doc)

for i, doc in enumerate(test_embedded_lst):
  test_embedded_lst[i] = eval(doc)

### Get train and test labels One-hot encodings

In [ ]:
train_labels_oh = pd.get_dummies(train['label-liar'])
test_labels_oh = pd.get_dummies(test['label-liar'])

train_labels = np.asarray(train_labels_oh).tolist()
test_labels = np.asarray(test_labels_oh).tolist()

### truncate/pad sequences to a *max sequence length*

First we need to determine the value of *max sequence length*.<br>
Find minimum, maximum and average sequence length.

In [ ]:
doc_len = len(test_embedded_lst[0])
min_len = doc_len
max_len = doc_len
avg_len = doc_len
for doc in test_embedded_lst:
  doc_len = len(doc)

  if min_len > doc_len:
    min_len = doc_len

  elif max_len < doc_len:
    max_len = doc_len
  
  avg_len = (avg_len+doc_len)/2

print('min sequence length =', min_len)
print('max sequence length =', max_len)
print('avrage sequence length =', avg_len)

min sequence length = 68
max sequence length = 102
avrage sequence length = 83.20663866249106


Decision: Our model will use a *max sequence length* of 230.

In [ ]:
def pad_list(lst, value=0, size=0):
  count=0
  while count < size:
    lst.append(value)
    count += 1

In [ ]:
maxlen = 230 # max sequence length

train_docs = []
for doc in train_embedded_lst:
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, value=[0]*5, size=pad_size) # pad end of seq with zeros
  train_docs.append(truncated_doc)

test_docs = []
for doc in test_embedded_lst:
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, value=[0]*5, size=pad_size) # pad end of seq with zeros
  test_docs.append(truncated_doc)

In [ ]:
# check that all sequence lengths' are the same
doc_lens = set()
for doc in train_docs:
  doc_lens.add(len(doc))

for doc in test_docs:
  doc_lens.add(len(doc))

print(doc_lens)

{230}


### Add positional embeddings

In [ ]:
def positional_embeddings(vector, dims):
  dim1, dim2 = dims
  pe = 0
  for pos in range(dim1):
    for i in range(dim2):

      if i%2 == 0: # is even
        pe = sin(pos/10000**(i/dim2))

      else: # is odd
        pe = cos(pos/10000**((i-1)/dim2))

      vector[pos][i] += pe

In [ ]:
word_size = 5 # each word is represented as a vector with dim = word_size

for vector in train_docs:
  positional_embeddings(vector, (maxlen, word_size))

for vector in test_docs:
  positional_embeddings(vector, (maxlen, word_size))

### Transformer block

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

### Build model and fit

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,5))
embedding_layer = layers.Dense(embed_dim) # regular fc layer
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(6, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
x_train = tf.convert_to_tensor(train_docs)
y_train = tf.convert_to_tensor(train_labels)

In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=["categorical_accuracy", tf.keras.metrics.Recall()])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_split=0.2, shuffle=True
)

Epoch 1/2
377/377 [==============================] - 113s 294ms/step - loss: 1.7654 - categorical_accuracy: 0.2194 - recall_24: 0.0011 - val_loss: 1.7134 - val_categorical_accuracy: 0.2547 - val_recall_24: 0.0017
Epoch 2/2
377/377 [==============================] - 113s 300ms/step - loss: 1.7246 - categorical_accuracy: 0.2507 - recall_24: 0.0057 - val_loss: 1.7146 - val_categorical_accuracy: 0.2617 - val_recall_24: 0.0169


### Evaluate model on test

In [ ]:
x_test = tf.convert_to_tensor(test_docs)
y_test = tf.convert_to_tensor(test_labels)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=32)

40/40 [==============================] - 4s 96ms/step - loss: 2.4300 - categorical_accuracy: 0.1643 - recall_24: 0.1619


## Model 3 - Binary classification, *max sequence length* = 512

In [ ]:
import pandas as pd
import numpy as np
import io
import time
from tqdm import tqdm
from math import sin, cos

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Get datasets

In [ ]:
#train
num_splits = 7
train = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/liar_train_{i}.csv')
  train = train.append(temp)

#train embedded
num_splits = 12
train_embedded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_embedded_v2_0.csv')
for i in range(1, num_splits):
  temp = pd.read_csv(f'https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/train_embedded_v2_{i}.csv')
  train_embedded = train_embedded.append(temp)

#test
test = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset-train/main/liar_test.csv')

#test embedded
test_embedded = pd.read_csv('https://raw.githubusercontent.com/AlonBrul/liar-liar-dataset/main/test_embedded.csv')

In [ ]:
train_embedded.head(3)

,full_text_embedded
0,"[[-0.37441736, -4.348481, -2.867022, -0.833148..."
1,"[[-1.0242298, -3.5270042, -1.9221768, 0.864446..."
2,"[[0.03841588, -3.0448728, -2.48761, 0.37428683..."


### Filter documents and labels where labels are binary ('true', 'false')

In [ ]:
train_labels_lst = train['label-liar'].tolist()
train_embedded_lst = train_embedded['full_text_embedded'].tolist()

test_labels_lst = test['label-liar'].tolist()
test_embedded_lst = test_embedded['full_text_embedded'].tolist()

In [ ]:
is_binary_train_index = lambda i: True if train_labels_lst[i] in ('true', 'false') else False

is_binary_test_index = lambda i: True if test_labels_lst[i] in ('true', 'false') else False

In [ ]:
train_labels_binary = []
train_embedded_binary = []

test_labels_binary = []
test_embedded_binary = []

for i in range(len(train_labels_lst)):
  if is_binary_train_index(i):
    train_labels_binary.append(train_labels_lst[i])
    train_embedded_binary.append(train_embedded_lst[i])

for i in range(len(test_labels_lst)):
  if is_binary_test_index(i):
    test_labels_binary.append(test_labels_lst[i])
    test_embedded_binary.append(test_embedded_lst[i])

In [ ]:
# documents embeddings are saved as strings
# use eval to return it to acctual embeddings (list of word embedding vectors)
for i, doc in enumerate(train_embedded_binary):
  train_embedded_binary[i] = eval(doc)

for i, doc in enumerate(test_embedded_binary):
  test_embedded_binary[i] = eval(doc)

### Get train and test labels One-hot encodings

In [ ]:
train_labels_binary_df = pd.DataFrame({'label-liar': train_labels_binary})
test_labels_binary_df = pd.DataFrame({'label-liar': test_labels_binary})

train_labels_oh = pd.get_dummies(train_labels_binary_df['label-liar'])
test_labels_oh = pd.get_dummies(test_labels_binary_df['label-liar'])

train_labels = np.asarray(train_labels_oh).tolist()
test_labels = np.asarray(test_labels_oh).tolist()

### truncate/pad sequences to a *max sequence length*

First we need to determine the value of *max sequence length*.<br>
Find minimum, maximum and average sequence length

In [ ]:
doc_len = len(train_embedded_binary[0])
min_len = doc_len
max_len = doc_len
avg_len = doc_len
for doc in train_embedded_binary:
  doc_len = len(doc)

  if min_len > doc_len:
    min_len = doc_len

  elif max_len < doc_len:
    max_len = doc_len
  
  avg_len = (avg_len+doc_len)/2

print('min sequence length =', min_len)
print('max sequence length =', max_len)
print('avrage sequence length =', avg_len)

min sequence length = 65
max sequence length = 291
avrage sequence length = 234.99282043613502


Decision: Our model will use a *max sequence length* of 230

In [ ]:
def pad_list(lst, value=0, size=0):
  count=0
  while count < size:
    lst.append(value)
    count += 1

In [ ]:
maxlen = 230 # max sequence length

train_docs = []
for doc in train_embedded_binary:
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, value=[0]*5, size=pad_size) # pad end of seq with zeros
  train_docs.append(truncated_doc)

test_docs = []
for doc in test_embedded_binary:
  truncated_doc = doc[:maxlen] # truncate to max len
  pad_size = maxlen - len(truncated_doc)
  pad_list(truncated_doc, value=[0]*5, size=pad_size) # pad end of seq with zeros
  test_docs.append(truncated_doc)

In [ ]:
# check that all sequence lengths' are the same
doc_lens = set()
for doc in train_docs:
  doc_lens.add(len(doc))

for doc in test_docs:
  doc_lens.add(len(doc))

print(doc_lens)

{230}


### Add positional embeddings

In [ ]:
def positional_embeddings(vector, dims):
  dim1, dim2 = dims
  pe = 0
  for pos in range(dim1):
    for i in range(dim2):

      if i%2 == 0: # is even
        pe = sin(pos/10000**(i/dim2))

      else: # is odd
        pe = cos(pos/10000**((i-1)/dim2))

      vector[pos][i] += pe

In [ ]:
word_size = 5 # each word is represented as a vector with dim = word_size

for vector in train_docs:
  positional_embeddings(vector, (maxlen, word_size))

for vector in test_docs:
  positional_embeddings(vector, (maxlen, word_size))

### Transformer block

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

### Build model and fit

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,5))
embedding_layer = layers.Dense(embed_dim) # (not realy an embedding_layer, regular fc layer)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
x_train = tf.convert_to_tensor(train_docs)
y_train = tf.convert_to_tensor(train_labels)

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["binary_accuracy", tf.keras.metrics.Recall()])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_split=0.2, shuffle=True
)

Epoch 1/2
134/134 [==============================] - 42s 302ms/step - loss: 0.6756 - binary_accuracy: 0.5956 - recall_25: 0.5956 - val_loss: 0.6406 - val_binary_accuracy: 0.6407 - val_recall_25: 0.6407
Epoch 2/2
134/134 [==============================] - 39s 295ms/step - loss: 0.6559 - binary_accuracy: 0.6173 - recall_25: 0.6173 - val_loss: 0.6389 - val_binary_accuracy: 0.6576 - val_recall_25: 0.6576


### Evaluate model on test

In [ ]:
x_test = tf.convert_to_tensor(test_docs)
y_test = tf.convert_to_tensor(test_labels)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=32)

15/15 [==============================] - 1s 97ms/step - loss: 0.7092 - binary_accuracy: 0.4902 - recall_25: 0.4902


In [ ]:
%%shell
jupyter nbconvert --to html /content/Fake_news_encoding.ipynb